In [1]:
# --- Step 1: Import libraries ---
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# --- Step 2: Load all CSVs automatically ---
file_paths = glob.glob("*.csv")  # adjust path if needed, e.g. "data/*.csv"

print("📂 Found CSV files:", file_paths)

dfs = []
for path in file_paths:
    df = pd.read_csv(path)
    
    # Add column indicating which CSV (study) this came from
    df["source"] = path.split("/")[-1]
    
    dfs.append(df)

# Combine all data
data = pd.concat(dfs, ignore_index=True)
print("✅ Total data shape:", data.shape)
print("✅ Sources found:", data['source'].unique())

# --- Step 3: Prepare text column ---
# Safely merge PhD and Postdoc columns regardless of capitalization

# Create merged PhD text
if {"phd", "PhD"}.issubset(data.columns):
    data["phd_merged"] = data[["phd", "PhD"]].fillna("").astype(str).agg(" ".join, axis=1)
elif "phd" in data.columns:
    data["phd_merged"] = data["phd"].fillna("").astype(str)
elif "PhD" in data.columns:
    data["phd_merged"] = data["PhD"].fillna("").astype(str)
else:
    data["phd_merged"] = ""

# Create merged Postdoc text
if {"postdoc", "Postdoc"}.issubset(data.columns):
    data["postdoc_merged"] = data[["postdoc", "Postdoc"]].fillna("").astype(str).agg(" ".join, axis=1)
elif "postdoc" in data.columns:
    data["postdoc_merged"] = data["postdoc"].fillna("").astype(str)
elif "Postdoc" in data.columns:
    data["postdoc_merged"] = data["Postdoc"].fillna("").astype(str)
else:
    data["postdoc_merged"] = ""

# Combine all text
data["text"] = (data["phd_merged"] + " " + data["postdoc_merged"]).str.strip()

# Drop rows with empty text
data = data[data["text"].str.strip() != ""]
print("✅ Example text data:")
print(data["text"].head())

# --- Step 4: Train-test split ---
X_train, X_test, y_train, y_test = train_test_split(
    data["text"], data["source"], test_size=0.2, random_state=42, stratify=data["source"]
)

# --- Step 5: TF-IDF vectorization ---
vectorizer = TfidfVectorizer(stop_words="english", max_features=3000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# --- Step 6: Train Random Forest ---
rf = RandomForestClassifier(
    n_estimators=200,        # number of trees
    random_state=42,
    n_jobs=-1,
    max_depth=None
)
rf.fit(X_train_tfidf, y_train)

# --- Step 7: Evaluate ---
y_pred = rf.predict(X_test_tfidf)

print("\n🎯 Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))

# --- Step 8: Try a prediction ---
sample = ["PhD (Biotechnology), Interdisciplinary Biotechnology Unit (Aligarh Muslim University), 2008, Specialization in Molecular Immunology"]
sample_tfidf = vectorizer.transform(sample)
predicted_source = rf.predict(sample_tfidf)[0]
print("\n🔮 Predicted CSV (source):", predicted_source)


📂 Found CSV files: ['iiserkol_facultybio.csv', 'nisersms_faculty.csv', 'niserscs_faculty.csv', 'iiserkol_facultymath.csv', 'iiserkol_facultychem.csv', 'nisersps_faculty.csv', 'iiserb_physics_faculty_full.csv', 'nisersbs_faculty.csv', 'iiserkol_facultyphy.csv', 'iiserbmath_faculty_phd_postdoc.csv', 'faculty_bhopbio.csv', 'faculty_with_phd.csv', 'iiserbpr_faculty.csv', 'iiserbchem_fac.csv']
✅ Total data shape: (803, 21)
✅ Sources found: ['iiserkol_facultybio.csv' 'nisersms_faculty.csv' 'niserscs_faculty.csv'
 'iiserkol_facultymath.csv' 'iiserkol_facultychem.csv'
 'nisersps_faculty.csv' 'iiserb_physics_faculty_full.csv'
 'nisersbs_faculty.csv' 'iiserkol_facultyphy.csv'
 'iiserbmath_faculty_phd_postdoc.csv' 'faculty_bhopbio.csv'
 'faculty_with_phd.csv' 'iiserbpr_faculty.csv' 'iiserbchem_fac.csv']
✅ Example text data:
0                  PhD (Biochemistry), IIT Delhi, 1990
1    PhD (Biotechnology), Interdisciplinary Biotech...
2    PhD (Biophysical Chemistry), Bose Institute (J...
3    PhD (

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
